In [1]:
import torch
import torch.nn as nn
from torchvision.models import resnet18

![Original-ResNet-18-Architecture.png](Original-ResNet-18-Architecture.png)

In [2]:
orig_model = resnet18(pretrained=True)
orig_model.eval()

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/kamikazi/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [82]:
REDUCTION_RATES = [1, 2, 4, 5, 6, 7, 8]
rank_map = {}
dummy_input = torch.rand(1, 3, 224, 224)

# original macs = C_i * W_k * H_k * C_o * W_o * H_o
# reduced macs = rank * C_i * W_i * H_i + rank**2 * W_k * H_k * W_o * H_o + rank * C_o * W_o * H_o
layer = orig_model.conv1
weight = orig_model.conv1.weight.detach()
bias = orig_model.conv1.bias
with torch.no_grad(): y = layer(dummy_input)
c1 = layer.in_channels * dummy_input.shape[-1] * dummy_input.shape[-2]
c2 = layer.kernel_size[0] * layer.kernel_size[1] * y.shape[-1] * y.shape[-2]
c3 = layer.out_channels * y.shape[-1] * y.shape[-2]
print('Layer: conv1')
print('bias:', bias is not None)
print('Original Shape:', weight.shape)
weight = weight.reshape((weight.shape[0], weight.shape[1], -1))
print('Flattened Shape:', weight.shape)
print('Parameters:', weight.numel())
orig_macs = torch.tensor(weight.shape).prod() * y.shape[-1] * y.shape[-2]
print('Original MACs:', orig_macs)
# print('Feasible ranks <', X.numel()/sum(list(X.shape))/REDUCTION_RATE)
ranks = list(int(weight.numel()/sum(list(weight.shape))/rate) for rate in REDUCTION_RATES)
print('Feasible ranks:', ranks)
# macs = [((rank * c1 + rank**2 * c2 + rank * c3) * 100/orig_macs).item() for rank in ranks]
# print('Feasible MACs(%):', [round(mac, 2) for mac in macs])
print()
rank_map['conv1'] = ranks
y = orig_model.maxpool(y)

for layer_name in ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 
                   'layer2.0.conv1', 'layer2.0.conv2', 'layer2.0.downsample', 'layer2.1.conv1', 'layer2.1.conv2',
                   'layer3.0.conv1', 'layer3.0.conv2', 'layer3.0.downsample', 'layer3.1.conv1', 'layer3.1.conv2',
                   'layer4.0.conv1', 'layer4.0.conv2', 'layer4.0.downsample', 'layer4.1.conv1', 'layer4.1.conv2']:
    lname, lidx, ltype = layer_name.split('.')
    lidx = int(lidx)
    layer = orig_model.__getattr__(lname)[lidx].__getattr__(ltype)
    if ltype == 'downsample': layer = layer[0]
    weight = layer.weight.detach()
    bias = orig_model.conv1.bias
    print('Layer:', layer_name)
    print('bias:', bias is not None)
    print('Original Shape:', weight.shape)
    weight = weight.reshape((weight.shape[0], weight.shape[1], -1))
    if ltype == 'downsample': weight = weight.reshape((weight.shape[0], weight.shape[1]))
    print('Flattened Shape:', weight.shape)
    
    print('Input shape:', y.shape)
    c1 = layer.in_channels * y.shape[-1] * y.shape[-2]
    if layer_name.endswith('.0.conv1'): 
        with torch.no_grad(): y = layer(y)
    c2 = layer.kernel_size[0] * layer.kernel_size[1] * y.shape[-1] * y.shape[-2]
    c3 = layer.out_channels * y.shape[-1] * y.shape[-2]
    print('Output shape:', y.shape)
    
    print('Parameters:', weight.numel())
    orig_macs = torch.tensor(weight.shape).prod() * y.shape[-1] * y.shape[-2]
    print('Original MACs:', orig_macs)
    ranks = list(int(weight.numel()/sum(list(weight.shape))/rate) for rate in REDUCTION_RATES)
    print('Feasible ranks:', ranks)
    if ltype != 'downsample': 
        macs = [((rank * c1 + rank**2 * c2 + rank * c3) * 100/orig_macs).item() for rank in ranks]
        print('Feasible MACs(%):', [round(mac, 2) for mac in macs])
    print()
    rank_map[layer_name] = ranks
    

# X = orig_model.fc.weight.detach()
# bias = orig_model.conv1.bias
# print('Layer: fc')
# print('Original Shape:', X.shape)
# print('Parameters:', X.numel())
# print('bias:', bias is not None)
# # print('Feasible rank < ', X.numel()/sum(list(X.shape))/rate)
# ranks = list(int(X.numel()/sum(list(X.shape))/rate) for rate in REDUCTION_RATES)
# print('Feasible ranks = ', ranks)
# rank_map['fc'] = ranks

Layer: conv1
bias: False
Original Shape: torch.Size([64, 3, 7, 7])
Flattened Shape: torch.Size([64, 3, 49])
Parameters: 9408
Original MACs: tensor(118013952)
Feasible ranks: [81, 40, 20, 16, 13, 11, 10]

Layer: layer1.0.conv1
bias: False
Original Shape: torch.Size([64, 64, 3, 3])
Flattened Shape: torch.Size([64, 64, 9])
Input shape: torch.Size([1, 64, 56, 56])
Output shape: torch.Size([1, 64, 56, 56])
Parameters: 36864
Original MACs: tensor(115605504)
Feasible ranks: [269, 134, 67, 53, 44, 38, 33]
Feasible MACs(%): [1860.03, 484.91, 132.86, 86.98, 62.54, 48.45, 38.05]

Layer: layer1.0.conv2
bias: False
Original Shape: torch.Size([64, 64, 3, 3])
Flattened Shape: torch.Size([64, 64, 9])
Input shape: torch.Size([1, 64, 56, 56])
Output shape: torch.Size([1, 64, 56, 56])
Parameters: 36864
Original MACs: tensor(115605504)
Feasible ranks: [269, 134, 67, 53, 44, 38, 33]
Feasible MACs(%): [1860.03, 484.91, 132.86, 86.98, 62.54, 48.45, 38.05]

Layer: layer1.1.conv1
bias: False
Original Shape: to

In [5]:
rank_map

{'conv1': [81, 40, 20, 10],
 'layer1.0.conv1': [269, 134, 67, 33],
 'layer1.0.conv2': [269, 134, 67, 33],
 'layer1.1.conv1': [269, 134, 67, 33],
 'layer1.1.conv2': [269, 134, 67, 33],
 'layer2.0.conv1': [366, 183, 91, 45],
 'layer2.0.conv2': [556, 278, 139, 69],
 'layer2.0.downsample': [42, 21, 10, 5],
 'layer2.1.conv1': [556, 278, 139, 69],
 'layer2.1.conv2': [556, 278, 139, 69],
 'layer3.0.conv1': [750, 375, 187, 93],
 'layer3.0.conv2': [1132, 566, 283, 141],
 'layer3.0.downsample': [85, 42, 21, 10],
 'layer3.1.conv1': [1132, 566, 283, 141],
 'layer3.1.conv2': [1132, 566, 283, 141],
 'layer4.0.conv1': [1518, 759, 379, 189],
 'layer4.0.conv2': [2283, 1141, 570, 285],
 'layer4.0.downsample': [170, 85, 42, 21],
 'layer4.1.conv1': [2283, 1141, 570, 285],
 'layer4.1.conv2': [2283, 1141, 570, 285],
 'fc': [338, 169, 84, 42]}